In [ ]:
import numpy as np

def extract_zeros(matrix):
    memo = {}
    for row in range(matrix.shape[0]):
        for col in range(matrix.shape[1]):
            if matrix[row,col] == 0:
                row_zeros = np.count_nonzero(matrix[row,:]==0)-1
                col_zeros = np.count_nonzero(matrix[:,col]==0)-1
                memo[(row,col)] = row_zeros + col_zeros
    locations = sorted(memo.items(), key=lambda item: item[1])[:len(matrix)]
    locations = [_[0] for _ in locations]
    return locations

def check(n, locations):
    rows = np.ones(n)
    cols = np.ones(n)
    for loc in locations:
        rows[loc[0]] = 0
        cols[loc[1]] = 0
    return not any(rows) and not any(cols)

def lines(matrix):
    cmatrix = matrix.copy()
    row_memo = {}
    col_memo = {}
    for i in range(len(matrix)):
        this_row_zeros = np.count_nonzero(matrix[i,:]==0)
        this_col_zeros = np.count_nonzero(matrix[:,i]==0)
        row_memo[i] = this_row_zeros
        col_memo[i] = this_col_zeros
    
    row_memo = sorted(row_memo.items(), key=lambda item: item[1])
    col_memo = sorted(col_memo.items(), key=lambda item: item[1])
    row_lines = []
    col_lines = []
    while 0 in cmatrix:
        if row_memo[-1][1] > col_memo[-1][1]:
            cmatrix[row_memo[-1][0], :] = 1
            row_lines.append(row_memo[-1][0])
            row_memo.pop()            
        else:
            cmatrix[:, col_memo[-1][0]] = 1
            col_lines.append(col_memo[-1][0])
            col_memo.pop()
    
    no_line_matrix = matrix.copy()
    for line in row_lines:
        no_line_matrix = np.delete(no_line_matrix, line, 0)
    for line in col_lines:
        no_line_matrix = np.delete(no_line_matrix, line, 1)

    min_element = np.min(no_line_matrix)
    min_matrix = -min_element * np.ones(matrix.shape)
    for line in row_lines:
        min_matrix[line, :] = 0
    for line in col_lines:
        min_matrix[:, line] = 0
    for row in row_lines:
        for col in col_lines:
            min_matrix[row][col] = min_element
    return matrix+min_matrix

def hungarian(matrix):
    org_matrix = matrix.copy()
    print(matrix)

    row_min = np.min(matrix, axis=1)
    matrix = (matrix.T - row_min).T
    print(matrix)

    locations = extract_zeros(matrix)
    if not check(len(matrix), locations):
        col_min = np.min(matrix, axis=0)
        matrix = matrix - col_min
        print(matrix)
        locations = extract_zeros(matrix)
        while not check(len(matrix), locations):
            matrix = lines(matrix)
            print(matrix)
            locations = extract_zeros(matrix)

    ans = 0
    for loc in locations:
        ans += org_matrix[loc]
    print("solved, answer=", ans)
    return matrix

from tkinter import *
from tkinter.ttk import *
import tkinter as tk
from PIL import Image, ImageTk


def show_entry_fields(entries):
    matrix = [[0]*3 for _ in range(3)]
    c = 0
    for i in range(3):
        for j in range(3):
            matrix[i][j] = int(entries[j+c].get())
        c += 3
    matrix = hungarian(np.array(matrix))
    c = 0
    for i in range(3):
        for j in range(3):
                entries[c].delete(0,END)
                entries[c].insert(0, matrix[i][j])
                c += 1


                
bg = '#DBCBDF'
                
master = tk.Tk()
tk.Label(master, 
         text="Enter a Matrix: ", fg="black", bg=bg, font=("Lato", 13)).grid(row=0,
                                                                             pady=5, padx=10)
tk.Label(master, 
         text="First row", bg=bg, font =("Lato", 11)).grid(row=1)
tk.Label(master, 
         text="Second row", bg=bg, font =("Lato", 11)).grid(row=2)
tk.Label(master, 
         text="Third row", bg=bg, font =("Lato", 11)).grid(row=3)

master.configure(bg=bg)
master.title("Hungarian Algorithm")
master.geometry('640x320')

style = Style()
style.configure('TButton', font =
               ('calibri', 15, 'bold'),
                    borderwidth = '1')
style.map('TButton', foreground = [('active', '!disabled', '#223344')],
                     background = [('active', '#101935')])


entries = [tk.Entry(master) for _ in range(9)]

entries[0].grid(row=1, column=1)
entries[1].grid(row=2, column=1)
entries[2].grid(row=3, column=1)
entries[3].grid(row=1, column=2)
entries[4].grid(row=2, column=2)
entries[5].grid(row=3, column=2)
entries[6].grid(row=1, column=3)
entries[7].grid(row=2, column=3)
entries[8].grid(row=3, column=3)


Button(master, text='Quit', command=master.destroy).grid(row=6, column=3,
                                                         sticky=tk.W,
                                                         padx=10, pady=2)
Button(master, text='Show', 
       command=lambda x=None: show_entry_fields(entries)).grid(row=6, column=2,
                                                               sticky=tk.W,
                                                               padx=10, pady=20)

font_size = 40

for e in entries:
    e.config(font=('Ink Free',font_size))
    e.config(bg='#223344')
    e.config(fg='#FFF')
    width = 5
    e.config(width=width)

tk.mainloop()